# Installing python packages into Notebook

In [ ]:
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install pyyaml
!{sys.executable} -m pip install oyaml
!{sys.executable} -m pip install pandas

In [ ]:
import api

In [ ]:
import imp
imp.reload(api)

# Runtime Review

In [ ]:
run = api.Runtime()

In [ ]:
api.Runtime.zones

## Load up data

In [ ]:
%%time
run = run.refresh()

## Summary Stats

In [ ]:
api.Table(run.stats())

### Alert Count

In [ ]:
sum([a["Num Links"] for a in run.stats()])

## All the links

In [ ]:
api.LinkTable(run.iterlinks())

In [ ]:
for zone in run.iterzones():
    for app in zone.iterapps():
        for link in app.iterlinks():
            print(link)

## Meta Policies and suggestions

In [ ]:
api.mpr.run(run.iterlinks()).review()

## Relocation Example

In [ ]:
api.LinkTable(run.iterzones("nightly").iterapps("bendvm").relocate("prod", "bendvm").iterlinks())